In [ ]:
from datetime import timedelta

from dydx_v4_client.indexer.rest.indexer_client import IndexerClient  # type: ignore
from dydx_v4_client.network import make_mainnet  # type: ignore
from sqlalchemy import create_engine  # type: ignore


from locast.candle_fetcher.dydx.api_fetcher.dydx_v4_fetcher import DydxV4Fetcher
from locast.candle_fetcher.dydx.candle_fetcher.dydx_v4_candle_fetcher import (
    DydxV4CandleFetcher,
)
from locast.store_manager.store_manager import StoreManager

from locast.candle.candle_utility import CandleUtility as cu
from locast.candle.dydx.dydx_resolution import DydxResolution
from locast.candle.exchange import Exchange
from locast.candle_storage.sql.sqlite_candle_storage import SqliteCandleStorage

MAINNET = make_mainnet(  # type: ignore
    rest_indexer="https://indexer.dydx.trade/",
    websocket_indexer="wss://indexer.dydx.trade/v4/ws",
    node_url="dydx-ops-rpc.kingnodes.com:443",
)

DELETE_CLUSTER = False


async def dydx_v4_example() -> None:
    # Create store manager.
    storage = SqliteCandleStorage(create_engine("sqlite:///locast.db"))  # type: ignore
    fetcher = DydxV4CandleFetcher(DydxV4Fetcher(IndexerClient(MAINNET.rest_indexer)))  # type: ignore
    manager = StoreManager(candle_fetcher=fetcher, candle_storage=storage)

    # Define desired candle cluster parameters.
    exchange = Exchange.DYDX_V4
    resolution = DydxResolution.ONE_MINUTE
    market = "ETH-USD"

    # Define start date for candle cluster.
    now = cu.normalized_now(resolution)  # Now, rounded down to nearest resolution tick
    start_date = now - timedelta(seconds=resolution.seconds * 5000)  # 5K candles back

    # Check for existing cluster.
    eth_info = await storage.get_cluster_info(exchange, market, resolution)

    # If cluster head not present (meaning: There is no candle cluster in the database for this market),
    # then create cluster (download from exchange & store in database).
    if not eth_info.head:
        print("Creating cluster...")
        await manager.create_cluster(market, resolution, start_date)

    # Check if cluster is up to date (for example if this notebook is run again later).
    # If not, update cluster.
    eth_info = await storage.get_cluster_info(exchange, market, resolution)
    if not eth_info.is_uptodate:
        print("Updating cluster...")
        await manager.update_cluster(exchange, market, resolution)
    else:
        next_tick = cu.next_tick(resolution)
        print(f"Cluster is up to date. Try again at {next_tick}.")

    # Check how many candles there are in this cluster
    eth_info = await storage.get_cluster_info(exchange, market, resolution)
    print(f"{market}-Candles in cluster: {eth_info.amount}.")

    # Set flag to True at top of this notebook to try out deleting the eth candle cluster
    if DELETE_CLUSTER:
        print("Deleting cluster...")
        await manager.delete_cluster(exchange, market, resolution)
        if cluster_info := await storage.get_cluster_info(exchange, market, resolution):
            print(f"{market}-Candles in cluster: {cluster_info.amount}.")


await dydx_v4_example()